Prepare the execution environment

In [1]:
import osmnx as ox
from gridding_helpers import Grid_Generation_Task_Manager, generate_grid, generate_file_name
from setting_helpers import load_yaml_config_file

Now load the settings from the settings yaml file for the area (lake) of interest:

In [2]:
settings_yaml_filepath = './settings/settings_talsperre_malter.yaml'
settings = load_yaml_config_file(settings_yaml_filepath)

tile_edge_lengths = settings['sensor_line_length_meter']
file_name = settings['geojson_file_name']
polygon_threshold = settings['polygon_threshold']

### How To find the lake or area of interest?!
First you need to find the area of interest...
Lets use the OSMnx module to get data through the OSM Overpass API.

Either use the OSM ID to find the geometry:

In [ ]:
malter_area = ox.geocode_to_gdf(query=["R2141671"], by_osmid=True)  # river dam Malter in Saxony, Germany
malter_area.head(1).geometry[0]

or build a search query and feed it to the OSMnx place search:

In [4]:
# place = "Talsperre Malter, Saxony, Germany"
# tags = {'water': True}
# ox_test = ox.geometries_from_place(place, tags)
# ox_test.head()

Create areas to examine as Polygons for every sensor scanner width with a Task Manager.
This uses ipyleaflet to draw interactive Maps.

If the saved areas overlap between smaller and greater widths: The greater width gets prioritized.

Create a Task Manager with the region of interest as area specification (must be MultiPolygon):

In [5]:
task_manager = Grid_Generation_Task_Manager(tile_edge_lengths, polygon_threshold, malter_area.head(1).geometry[0], file_name)

Wanna set the area for one sensor scanner width? Show its Map for the interactive MultiPolygon draw.
If you don't draw anything for this scanner width there will be no restrictions and the grid generation tries to fill the whole region of interest.

In [ ]:
one_task = task_manager.set_task_area(tile_edge_lengths[-1])
one_task.task_map

Confirm all chosen Polygons for this sensor scanner width.
You MUST always do this if you want to set specific areas to examine.
This either confirms your drawn areas or applies the whole region of interest for this scanner width.

In [ ]:
one_task.confirm_choices()

Specify the area for every scanner width you need.

In [ ]:
one_task = task_manager.set_task_area(tile_edge_lengths[-2])
one_task.task_map

In [ ]:
one_task.confirm_choices()  # don't forget to save your specifications

If you are done with all your area specifications (or you don't need to define specific areas for every sensor scanner width)
you can use the generate_grid method and start the grid generation process with giving it the task manager.

In [ ]:
grid_gdf = generate_grid(task_manager)

if not grid_gdf.empty:
    # save best results
    file_name = generate_file_name(settings['geojson_file_name'])
    grid_gdf.to_file(filename=f'./geodataframes/{file_name}_grid.geojson', driver="GeoJSON")
    print("Successfully finished grid generation and saved geometry to file!\n",
          f'./geodataframes/{file_name}_grid.geojson')